In [5]:
import ftplib
f = ftplib.FTP("10.21.3.24")  # 瀹炰緥鍖朏TP瀵硅薄
f.login("ftpuser", "pwd")  # 鐧诲綍
def ftp_download(remotename,localname):
    file_remote = "tiku/images/"+remotename
    file_local = "/data/fjs/data/math/"+localname
    bufsize = 1024  # 璁剧疆缂撳啿鍣ㄥぇ灏廫n    fp = open(file_local, 'wb')
    f.retrbinary('RETR %s' % file_remote, fp.write, bufsize)
    fp.close()
remotename="upimages/images/part/8730D.png"
localname="8730D.png"
ftp_download(remotename,localname)
f.quit()

'221 Goodbye.'

In [58]:
import pymysql
from html.parser import HTMLParser
import re
import string
import json

#html瑙ｆ瀽
class mathHTMLParser(HTMLParser): 
    def __init__(self): 
        HTMLParser.__init__(self) 
        self.texts = [] #闈炵粨鏋勫寲-鏂囨湰
        self.images = [] #闈炵粨鏋勫寲-鍥剧墖
        #self.htmls = [] #鍗婄粨鏋勫寲-html鏍煎紡
        #self.latexs = [] #鍗婄粨鏋勫寲-latex鏍煎紡
        
    def handle_starttag(self, tag, attrs):
        if tag=='img':
            for a in attrs:
                if a[0] == 'src':
                    self.images.append(a[1])
        if tag=='div':
            for a in attrs:
                if a[0]=='style':
                    if a[1].find('url')!=-1:
                        self.images.append(a[1].split("'")[1])

    def handle_endtag(self, tag):
        pass

    def handle_data(self, data):
        data = re.sub('[%s]' % re.escape(string.punctuation), '', data)
        if data.count('\n') == 0 and data!='':
            self.texts.append(data)
            
#mysql 
conn = pymysql.connect(host='10.21.3.23', user='name', passwd="pwd", db='CNK12')
cur = conn.cursor()
#鏁板瀛︾灏忓瀛︽913851鏉★紝鍒濅腑1202389锛岄珮涓�1507119
items=[]#list锛屼繚瀛樿В鏋愬悗鐨勬潯鐩甛nfor pagesize in range(0,18):#鎻愬彇灏忓瀛︽90涓囨潯鏁板涔犻锛宮ysql鎬ц兘浣庯紝閲囩敤limit鍒嗗竷鏌ヨ
    sel_sql ="select distinct(a.question_uid),a.question_diff,a.question_knowledge,\
                 a.question_text,a.question_options,a.question_answer,a.question_explain,a.question_parse \
                 from cnk_question a \
                 left join cnk_knowledge_question b on (a.question_uid=b.question_uid) \
                 left join cnk_knowledge c on (b.knowledge_uid=c.knowledge_uid and c.subject_id=2 and c.pharse_id=1)\
                 limit %d,50000"%(pagesize*50000)
    cur.execute(sel_sql)
    results = cur.fetchall()
    for row in results:
        mhp = mathHTMLParser() 
        item={}#dict
        item['quid']=row[0]
        item['qdiff']=row[1]
        item['qklg']=row[2]
        mhp.feed(row[3])#棰樺共
        mhp.feed(row[4])#閫夐」
        mhp.feed(row[5])#绛旀
        mhp.feed(row[6])#瑙ｉ噴
        mhp.feed(row[7])#鍒嗘瀽
        item['qtext']=mhp.texts  #闈炵粨鏋勫寲-鏂囨湰
        item['qimage']=mhp.images #闈炵粨鏋勫寲-鍥剧墖
        #item['qhtml'] =mhp.htmls #鍗婄粨鏋勫寲-html鏍煎紡
        #item['qlatex']=mhp.latexs #鍗婄粨鏋勫寲-latex鏍煎紡
        mhp.close() 
        items.append(item)#鎻掑叆list
    print ("page %d parsing completed."% pagesize)
cur.close()
conn.close()
print (len(items))#鎬婚暱搴nwith open("/data/fjs/data/math/math.json","w") as dump_f:
    json.dump(items,dump_f)#淇濆瓨鍒癹son鏂囦欢
    print("Json file saving completed.")

page 0 parsing completed.
page 1 parsing completed.
page 2 parsing completed.
page 3 parsing completed.
page 4 parsing completed.
page 5 parsing completed.
page 6 parsing completed.
page 7 parsing completed.
page 8 parsing completed.
page 9 parsing completed.
page 10 parsing completed.
page 11 parsing completed.
page 12 parsing completed.
page 13 parsing completed.
page 14 parsing completed.
page 15 parsing completed.
page 16 parsing completed.
page 17 parsing completed.
900000
Json file saving completed.


In [59]:
from itertools import compress
import pandas as pd
from operator import itemgetter
from itertools import groupby
#鍔熻兘锛歭ist閲岄潰鐨勬瘡涓�涓厓绱犻兘鏄痙ict锛屾牴鎹甦ict鏌愪竴涓猭ey杩涜鍘婚噸
#鍑芥暟1
def distinct(items,key):
    key = itemgetter(key)
    items = sorted(items, key=key)
    return [next(v) for _, v in groupby(items, key=key)]
#鍑芥暟2
#def distinct(items,key):  
#    mask = (~pd.Series(map(itemgetter(key), items)).duplicated()).tolist()
#    return list(compress(items, mask))
with open("/data/fjs/data/math/math.json",'r') as load_f:
    items = json.load(load_f)
    print (len(items))#鍘婚噸鍓嶆潯鐩甛n    items =distinct(items,'quid')#鍘婚噸key=quid
    print (len(items))#鍘婚噸鍚庢潯鐩�

900000
900000
